<a href="https://colab.research.google.com/github/Akanksha-Bathla/Akanksha-Bathla/blob/main/Audd_multi_modality_for_urdu_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [14]:
!wget -O UrbanSound8K.tar.gz https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
!tar -xzf UrbanSound8K.tar.gz

--2024-12-03 06:26:25--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1203745/files/UrbanSound8K.tar.gz [following]
--2024-12-03 06:26:26--  https://zenodo.org/records/1203745/files/UrbanSound8K.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘UrbanSound8K.tar.gz’

UrbanSound8K.tar.gz 100%[===================>]   5.61G  21.0MB/s    in 4m 34s  

2024-12-03 06:31:00 (21.0 MB/s) - ‘UrbanSound8K.tar.gz’ saved [6023741708/6023741708]



In [5]:
# PROCESSING
# Feature extraction methods

def CalculateZeroCrossingRate(file_location):
    y, sr = librosa.load(file_location)
    zCross = librosa.feature.zero_crossing_rate(y=y)
    dim = (32, 32)
    resized = cv2.resize(zCross, dim, interpolation = cv2.INTER_AREA)
    return resized

def CalculateMelSpectrogram(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr)
    melSpec_dB = librosa.power_to_db(melSpec)
    dim = (32, 32)
    resized = cv2.resize(melSpec_dB, dim, interpolation = cv2.INTER_AREA)
    return resized


def CalculateMFCC(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.mfcc(y=y, sr=sr)
    dim = (32, 32)
    resized = cv2.resize(melSpec, dim, interpolation = cv2.INTER_AREA)
    return resized



In [7]:
cd /content

/content


In [ ]:
'''for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue
      print(t.split("WA")[1].split(".")[0])
      #print(t.split("D")[1][0])
      break
    break
  break '''

['7', '8', '1', '6', '3', '2', '5', '4', '0', '9']
7
6614


In [16]:
import os
import numpy as np

# Define paths
files_path = '/content/drive/MyDrive/processed_data/audd'
if not os.path.exists(files_path):
    os.makedirs(files_path)

x = []
y = []
total = 23500  # Total number of expected files (if known)
count = 0

# Main directory containing the 'fold' subdirectories
main_dir = '/content/UrbanSound8K/audio'

# List all subdirectories (fold1, fold2, etc.)
for fold in os.listdir(main_dir):
    fold_path = os.path.join(main_dir, fold)
    print(f"Processing folder: {fold}")

    # Ensure it's a directory
    if not os.path.isdir(fold_path):
        continue

    # Process each file in the fold directory
    for file_name in os.listdir(fold_path):
        if not file_name.endswith('.wav'):  # Process only .wav files
            continue

        print(count, file_name)
        file_path = os.path.join(fold_path, file_name)

        # Extract features from the .wav file (replace with your logic)
        x.append(CalculateZeroCrossingRate(file_path))  # Or CalculateMelSpectrogram(), CalculateMFCC()

        # Assign labels (you might need to extract labels based on a convention, e.g., file name)
        # Here, labels are derived from the folder name or file-specific metadata.
        y.append(int(fold.replace('fold', '')))  # Example: convert "fold1" -> 1

        count += 1
        if count % 300 == 0:
            print(f"Remaining files: {total - count}")

# Convert lists to NumPy arrays
x = np.array(x)
y = np.array(y)

# Save the arrays to .npy files
np.save(os.path.join(files_path, "X_zero_crossing_rate"), x)
np.save(os.path.join(files_path, "y_zero_crossing_rate"), y)

print(f"Processing completed. Saved processed data in {files_path}.")


Streaming output truncated to the last 5000 lines.
3756 102853-8-0-0.wav
3757 102853-8-0-6.wav
3758 14772-7-0-0.wav
3759 189981-0-0-6.wav
3760 6902-2-0-7.wav
3761 102853-8-1-0.wav
3762 118587-3-0-2.wav
3763 164797-2-0-29.wav
3764 146845-0-0-8.wav
3765 7060-6-1-0.wav
3766 158593-2-0-54.wav
3767 39857-5-1-3.wav
3768 104625-4-0-15.wav
3769 71529-4-1-6.wav
3770 89099-4-0-0.wav
3771 107357-8-1-4.wav
3772 177729-0-0-20.wav
3773 146845-0-0-30.wav
3774 201988-5-0-6.wav
3775 177537-7-0-15.wav
3776 77770-9-0-6.wav
3777 44737-5-0-3.wav
3778 6902-2-0-26.wav
3779 105289-8-2-3.wav
3780 177537-7-0-23.wav
3781 189981-0-0-16.wav
3782 57696-4-0-1.wav
3783 177537-7-0-13.wav
3784 116485-3-0-4.wav
3785 77751-7-8-0.wav
3786 189987-0-0-3.wav
3787 196070-2-0-0.wav
3788 177729-0-0-42.wav
3789 165192-9-0-118.wav
3790 135527-6-14-4.wav
3791 74965-4-0-1.wav
3792 146845-0-0-9.wav
3793 168846-5-0-0.wav
3794 209992-5-2-79.wav
3795 50629-4-4-9.wav
3796 174284-6-1-0.wav
3797 66619-2-0-5.wav
3798 196070-2-0-5.wav
3799 

In [18]:
import os
import numpy as np

# Define paths
files_path = '/content/drive/MyDrive/processed_data/audd'
if not os.path.exists(files_path):
    os.makedirs(files_path)

x = []
y = []
total = 23500  # Total number of expected files (if known)
count = 0

# Main directory containing the 'fold' subdirectories
main_dir = '/content/UrbanSound8K/audio'

# List all subdirectories (fold1, fold2, etc.)
for fold in os.listdir(main_dir):
    fold_path = os.path.join(main_dir, fold)
    print(f"Processing folder: {fold}")

    # Ensure it's a directory
    if not os.path.isdir(fold_path):
        continue

    # Process each file in the fold directory
    for file_name in os.listdir(fold_path):
        if not file_name.endswith('.wav'):  # Process only .wav files
            continue

        print(count, file_name)
        file_path = os.path.join(fold_path, file_name)

        # Extract features from the .wav file (replace with your logic)
        x.append(CalculateMelSpectrogram(file_path))  # Or CalculateMelSpectrogram(), CalculateMFCC()

        # Assign labels (you might need to extract labels based on a convention, e.g., file name)
        # Here, labels are derived from the folder name or file-specific metadata.
        y.append(int(fold.replace('fold', '')))  # Example: convert "fold1" -> 1

        count += 1
        if count % 300 == 0:
            print(f"Remaining files: {total - count}")

# Convert lists to NumPy arrays
x = np.array(x)
y = np.array(y)

# Save the arrays to .npy files
np.save(os.path.join(files_path, "X_melspectrogram"), x)
np.save(os.path.join(files_path, "y_melspectrogram"), y)

print(f"Processing completed. Saved processed data in {files_path}.")


Streaming output truncated to the last 5000 lines.
3756 102853-8-0-0.wav
3757 102853-8-0-6.wav
3758 14772-7-0-0.wav
3759 189981-0-0-6.wav
3760 6902-2-0-7.wav
3761 102853-8-1-0.wav
3762 118587-3-0-2.wav
3763 164797-2-0-29.wav
3764 146845-0-0-8.wav
3765 7060-6-1-0.wav
3766 158593-2-0-54.wav
3767 39857-5-1-3.wav
3768 104625-4-0-15.wav
3769 71529-4-1-6.wav
3770 89099-4-0-0.wav
3771 107357-8-1-4.wav
3772 177729-0-0-20.wav
3773 146845-0-0-30.wav
3774 201988-5-0-6.wav
3775 177537-7-0-15.wav
3776 77770-9-0-6.wav
3777 44737-5-0-3.wav
3778 6902-2-0-26.wav
3779 105289-8-2-3.wav
3780 177537-7-0-23.wav
3781 189981-0-0-16.wav
3782 57696-4-0-1.wav
3783 177537-7-0-13.wav
3784 116485-3-0-4.wav
3785 77751-7-8-0.wav
3786 189987-0-0-3.wav
3787 196070-2-0-0.wav
3788 177729-0-0-42.wav
3789 165192-9-0-118.wav
3790 135527-6-14-4.wav
3791 74965-4-0-1.wav
3792 146845-0-0-9.wav
3793 168846-5-0-0.wav
3794 209992-5-2-79.wav
3795 50629-4-4-9.wav
3796 174284-6-1-0.wav
3797 66619-2-0-5.wav
3798 196070-2-0-5.wav
3799 

In [19]:
import os
import numpy as np

# Define paths
files_path = '/content/drive/MyDrive/processed_data/audd'
if not os.path.exists(files_path):
    os.makedirs(files_path)

x = []
y = []
total = 23500  # Total number of expected files (if known)
count = 0

# Main directory containing the 'fold' subdirectories
main_dir = '/content/UrbanSound8K/audio'

# List all subdirectories (fold1, fold2, etc.)
for fold in os.listdir(main_dir):
    fold_path = os.path.join(main_dir, fold)
    print(f"Processing folder: {fold}")

    # Ensure it's a directory
    if not os.path.isdir(fold_path):
        continue

    # Process each file in the fold directory
    for file_name in os.listdir(fold_path):
        if not file_name.endswith('.wav'):  # Process only .wav files
            continue

        print(count, file_name)
        file_path = os.path.join(fold_path, file_name)

        # Extract features from the .wav file (replace with your logic)
        x.append(CalculateMFCC(file_path))  # Or CalculateMelSpectrogram(), CalculateMFCC()

        # Assign labels (you might need to extract labels based on a convention, e.g., file name)
        # Here, labels are derived from the folder name or file-specific metadata.
        y.append(int(fold.replace('fold', '')))  # Example: convert "fold1" -> 1

        count += 1
        if count % 300 == 0:
            print(f"Remaining files: {total - count}")

# Convert lists to NumPy arrays
x = np.array(x)
y = np.array(y)

# Save the arrays to .npy files
np.save(os.path.join(files_path, "X_mfcc"), x)
np.save(os.path.join(files_path, "y_mfcc"), y)

print(f"Processing completed. Saved processed data in {files_path}.")


Streaming output truncated to the last 5000 lines.
3756 102853-8-0-0.wav
3757 102853-8-0-6.wav
3758 14772-7-0-0.wav
3759 189981-0-0-6.wav
3760 6902-2-0-7.wav
3761 102853-8-1-0.wav
3762 118587-3-0-2.wav
3763 164797-2-0-29.wav
3764 146845-0-0-8.wav
3765 7060-6-1-0.wav
3766 158593-2-0-54.wav
3767 39857-5-1-3.wav
3768 104625-4-0-15.wav
3769 71529-4-1-6.wav
3770 89099-4-0-0.wav
3771 107357-8-1-4.wav
3772 177729-0-0-20.wav
3773 146845-0-0-30.wav
3774 201988-5-0-6.wav
3775 177537-7-0-15.wav
3776 77770-9-0-6.wav
3777 44737-5-0-3.wav
3778 6902-2-0-26.wav
3779 105289-8-2-3.wav
3780 177537-7-0-23.wav
3781 189981-0-0-16.wav
3782 57696-4-0-1.wav
3783 177537-7-0-13.wav
3784 116485-3-0-4.wav
3785 77751-7-8-0.wav
3786 189987-0-0-3.wav
3787 196070-2-0-0.wav
3788 177729-0-0-42.wav
3789 165192-9-0-118.wav
3790 135527-6-14-4.wav
3791 74965-4-0-1.wav
3792 146845-0-0-9.wav
3793 168846-5-0-0.wav
3794 209992-5-2-79.wav
3795 50629-4-4-9.wav
3796 174284-6-1-0.wav
3797 66619-2-0-5.wav
3798 196070-2-0-5.wav
3799 

In [21]:
#load files
files_path = '/content/drive/MyDrive/processed_data/audd'

x_melspectrogram = np.load(files_path+"/X_melspectrogram.npy", allow_pickle=True)
x_zero_crossing_rate = np.load(files_path+"/X_zero_crossing_rate.npy", allow_pickle=True)
x_mfcc = np.load(files_path+"/X_mfcc.npy", allow_pickle=True)
y_melspectrogram = np.load(files_path+"/y_melspectrogram.npy", allow_pickle=True)
y_zero_crossing_rate = np.load(files_path+"/y_zero_crossing_rate.npy", allow_pickle=True)
y_mfcc = np.load(files_path+"/y_mfcc.npy", allow_pickle=True)

In [22]:
#Split data into train and test

from sklearn.model_selection import train_test_split
x = x_mfcc
y = y_mfcc

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_mfcc_train",x_train)
np.save(files_path+"/y_mfcc_train",y_train)
np.save(files_path+"/x_mfcc_test",x_test)
np.save(files_path+"/y_mfcc_test",y_test)

In [23]:

from sklearn.model_selection import train_test_split
x = x_melspectrogram
y = y_melspectrogram

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_melspectrogram_train",x_train)
np.save(files_path+"/y_melspectrogram_train",y_train)
np.save(files_path+"/x_melspectrogram_test",x_test)
np.save(files_path+"/y_melspectrogram_test",y_test)

In [24]:

from sklearn.model_selection import train_test_split
x = x_zero_crossing_rate
y = y_zero_crossing_rate

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_zero_crossing_rate_train",x_train)
np.save(files_path+"/y_zero_crossing_rate_train",y_train)
np.save(files_path+"/x_zero_crossing_rate_test",x_test)
np.save(files_path+"/y_zero_crossing_rate_test",y_test)

In [ ]:
import tensorflow as tf
import numpy as np

files_path = '/content/drive/MyDrive/processed_data/audd'

features = ['mel_spectrogram', 'mfcc', 'zero_crossing_rate']

x_train = np.load(files_path+"/x_{}_train.npy".format(features[2]), allow_pickle=True)
y_train = np.load(files_path+"/y_{}_train.npy".format(features[2]), allow_pickle=True)

x_test = np.load(files_path+"/x_{}_test.npy".format(features[2]),allow_pickle=True)
y_test = np.load(files_path+"/y_{}_test.npy".format(features[2]),allow_pickle=True)

In [ ]:
print(np.unique(y_melspectrogram))

[0 1 2 3 4 5 6 7 8 9]


In [ ]:
x_train =x_train.reshape((x_train.shape[0],32,32,1))
x_test =x_test.reshape((x_test.shape[0],32,32,1))

## Train Model

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
def GetCNN():
  model = models.Sequential()

  model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Dropout(0.1))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))# fully connected
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.1))

  model.add(layers.Dense(10,activation='softmax', use_bias=True))
  return model

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
train_y = label_as_binary.fit_transform(y_train)
test_y = label_as_binary.fit_transform(y_test)

In [ ]:
save_path = '/content/drive/MyDrive/audd saved files/models'


from tensorflow.keras.callbacks import ModelCheckpoint
cnn_model = GetCNN()
print(cnn_model.summary())
cnn_model.compile(loss  = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ["accuracy"],
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
callbacks = ModelCheckpoint(save_path+"/melspectrogram_model_.h5", monitor='val_accuracy',
                                    mode='max',
                                    save_best_only=True,
                                    verbose=1)

print("X train shape = ",x_train.shape)
print("y train shape = ",train_y.shape)

cnn_model.fit(x_train, train_y, batch_size=64, epochs=150, verbose=1, validation_split=0.1, callbacks=[callbacks])
#import keras
#cnn_model = keras.models.load_model(save_path+"/melspectrogram_model_.h5")
#print("Test Accuracy is ", cnn_model.evaluate(x_test,test_y)[1])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 30, 30, 64)        640       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 15, 15, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 13, 13, 64)        36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                      

ValueError: ignored